In [1]:
!pip install torch torchvision opencv-python

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import Cityscapes
from torchvision.transforms import ToTensor, Compose, Normalize, Resize
from PIL import Image
import matplotlib.pyplot as plt

# 加载预训练的DeepLab模型
model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
model = model.to('cuda')
model.train()

# 数据预处理和加载
transform = Compose([
    Resize((512, 1024)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = Cityscapes(data_dir, split='train', mode='fine', target_type='semantic', transforms=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

# 定义优化器和损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

# 训练循环
for epoch in range(10):  # 假设训练10个epoch
    for images, targets in dataloader:
        images = images.to('cuda')
        targets = targets.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [ ]:
# 加载行车记录仪图像
img = Image.open("/content/drive/My Drive/YourDashboardImage.png")
input_tensor = transform(img).unsqueeze(0).to('cuda')

# 进行推理
model.eval()
with torch.no_grad():
    output = model(input_tensor)['out'][0]
output_predictions = output.argmax(0).cpu().numpy()

# 显示结果
plt.imshow(output_predictions)
plt.show()

In [ ]:
torch.save(model.state_dict(), "/content/drive/My Drive/deeplab_cityscapes.pth")